In [1]:
import numpy as np

In [ ]:
def init(gp_config):
    population = []
    def create_population():
        return 
    def evaluate_fitness():
        return
    def do_cross_over(parent1, parent2):
        return
    def do_mutation(ind):
        return
    def train():
        return
    return {
        'train':train
    }